In [2]:
#General
import os
import sys
from datetime import datetime 
from datetime import timedelta
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook

# Object Detection
import six.moves.urllib as urllib
import tarfile
import tensorflow as tf
import zipfile
from collections import defaultdict
from io import StringIO
from PIL import Image
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# if tf.__version__ < '1.4.0':
#   raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

In [3]:
# This is needed to display the images.
%matplotlib inline

In [4]:
# Set-up directory
# Memory stick = /Volumes/FRED SHONE
# Drive = Elements
# Test = 
stick_location = '/Volumes/FRED SHONE' # Memory Stick
drive_location = '/Volumes/Elements' # Drive
test0 = 'test_inputs/test0'
test1 = 'test_inputs/test1'
test2 = 'test_inputs/test2'
test3 = 'test_inputs/test3'

disk_path = test2

print(disk_path)

test_inputs/test2


In [5]:
# Access daily sub-directories of images

folders = os.listdir(disk_path) # Assumes sorting by name is ok - but this won't work for changes in year

# Filter for days by trying to convert to datetime

day_directories = []
dates = []
for folder in folders:
    try: 
        date = datetime.strptime(folder, '%d-%m-%Y')
        day_directories.append(folder)
        dates.append(date)
    except:
        pass
    
day_directories = [x for y, x in sorted(zip(dates, day_directories))]
    
print(day_directories)

['01-03-2015']


In [6]:
# Check for number of images in day directories

for day_directory in day_directories:
    day_dir_path = os.path.join(disk_path, day_directory)
    image_paths = os.listdir(day_dir_path)
    image_paths = sorted([image for image in os.listdir(day_dir_path) if not image.startswith('.')])
    image_count = len(image_paths)
    print('{}: {} images'.format(day_directory, image_count))   
    #Wilkins building listed as opening hours from 8am to 11pm = 15 hours
    frame_rate = image_count/(15*60*60)
    print('frame rate assumed to be ~{} frames per second'.format(frame_rate))

01-03-2015: 100 images
frame rate assumed to be ~0.001851851851851852 frames per second


In [7]:
model_zoo = {'sd_mobilenet_v1_coco' : 'ssd_mobilenet_v1_coco_2018_01_28',
            'sd_mobilenet_v2_coco' : 'ssd_mobilenet_v2_coco_2018_03_29',
            'ssdlite_mobilenet_v2_coco' : 'ssdlite_mobilenet_v2_coco_2018_05_09',
            'ssd_inception_v2_coco' : 'ssd_inception_v2_coco_2018_01_28',
            'faster_rcnn_inception_v2_coco' : 'faster_rcnn_inception_v2_coco_2018_01_28',
            'faster_rcnn_resnet50_coco' : 'faster_rcnn_resnet50_coco_2018_01_28',
            'faster_rcnn_resnet50_lowproposals_coco' : 'faster_rcnn_resnet50_lowproposals_coco_2018_01_28',
            'rfcn_resnet101_coco' : 'rfcn_resnet101_coco_2018_01_28',
            'faster_rcnn_resnet101_coco' : 'faster_rcnn_resnet101_coco_2018_01_28',
            'faster_rcnn_resnet101_lowproposals_coco' : 'faster_rcnn_resnet101_lowproposals_coco_2018_01_28'}

mask_zoo = {'mask_rcnn_inception_v2_coco' : 'mask_rcnn_inception_v2_coco_2018_01_28'}

In [8]:
model_choice = 'ssd_inception_v2_coco'

In [9]:
# Prepare paths
MODEL_NAME = model_zoo.get(model_choice)

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('object_detection', 'data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

# Load label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [10]:
#Load Frozen model into memory
def load_model(model_name, location = 'object_detection'):

    # Path to frozen detection graph. This is the actual model that is used for the object detection.
    ckpt_path = os.path.join(location, model_name, 'frozen_inference_graph.pb')
    
    # Download if necessary
    if not os.path.isfile(ckpt_path):
        print('>>> Cannot find frozen model')
        download_model(model_name, location)
    
    print('Loading frozen model: {}'.format(ckpt_path))
    detection_graph = tf.Graph()
    with detection_graph.as_default():
      od_graph_def = tf.GraphDef()
      with tf.gfile.GFile(ckpt_path, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
    print('Done')
    return detection_graph

In [11]:
# Download Model
def download_model(model_name, location = 'object_detection'):

    model_file = model_name + '.tar.gz'
    download_base = 'http://download.tensorflow.org/models/object_detection'
    download_path = os.path.join(download_base, model_file)
    model_path = os.path.join(location, model_file)
    
    # Download if necessary
    if not os.path.isfile(model_path):
        
        print('Downloading: {}'.format(model_file))
        opener = urllib.request.URLopener()
        opener.retrieve(download_path, model_path)
        
    print('Extracting frozen inference graph from: {}'.format(model_path))
    tar_file = tarfile.open(model_path)
    for file in tar_file.getmembers():
      file_name = os.path.basename(file.name)
      if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.path.join(os.getcwd(), location))

In [12]:
detection_graph = load_model(MODEL_NAME)

Loading frozen model: object_detection/ssd_inception_v2_coco_2018_01_28/frozen_inference_graph.pb
Done


In [13]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [14]:
# Set-up motion detector
sensitivity = 0.0001 # Total pixel difference to trigger motion detection
warning_trigger = 10000 # Total pixel difference to trigger verbose image plot

verbose = False 
start_time = '08:00' # Assumes camera turned on at 8am every day
frame_step = 3 # Seconds

detect_objects = True
detection_threshold = 0.5

In [15]:
def trigger_image(img):
    # Reduce image size to 1%
    thumb = cv2.resize(img, None, fx=.1, fy=.1, interpolation = cv2.INTER_AREA)
    thumb = cv2.cvtColor(thumb, cv2.COLOR_BGR2GRAY)
        
    return thumb

def motion_trigger(sensitivity, thumb, background):

    if background is None: # Initialise background for first pass
        background = thumb

    delta = cv2.absdiff(background, thumb)
    thresh = cv2.threshold(delta, 25, 255, cv2.THRESH_BINARY)[1]
    pixel_diff = cv2.sumElems(thresh)[0]
    
    if pixel_diff > thumb.size*sensitivity:
        return True
    else:
        return False


In [16]:
# Object Detection
def detection(sess, detection_graph, img, frame):

    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(img, axis=0)
    
    # Actual detection.
    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})

    # Select observations scored above threshold (default 0.5)
    boxes = boxes[scores>detection_threshold]
    classes = classes[scores>detection_threshold]
    scores = scores[scores>detection_threshold]
    num = len(classes)
    if num:
        detection_record = frame

    '''
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8)
        '''



In [17]:
def track(model_id, disk_path, day_directories):
    
    detection_graph = load_model(model_id)
    
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:

            for day_directory in tqdm_notebook(day_directories, desc='Days:'):

                day_dir_path = os.path.join(disk_path, day_directory)
                images = sorted([image for image in os.listdir(day_dir_path) if not image.startswith('.')])

                date = datetime.strptime(day_directory,'%m-%d-%Y')
                start_datetime = datetime.strptime(day_directory+' '+start_time,'%m-%d-%Y %H:%M')

                background = None # Initialise with no background

                for image in tqdm_notebook(images, desc='{} progress:'.format(date)):

                    image_path = os.path.join(day_dir_path, image)
                    img = cv2.imread(image_path)

                    # Extract frame number from image path
                    image_index = [int(s) for s in list(image) if s.isdigit()]
                    image_index = ''.join(str(x) for x in image_index) 
                    image_index = int(image_index)

                    # Calculate time stamp of frame
                    time_delta = timedelta(seconds=(frame_step*image_index))
                    time_stamp = start_datetime + time_delta
                    
                    # Trigger ##############################################
                    thumb =  trigger_image(img)
                    
                    if motion_trigger(sensitivity, thumb, background):
                        
                        detection(sess, detection_graph, img, time_stamp)

                    background = thumb # Set background for next frame
                    

In [18]:

for model_name, model_id in model_zoo.items():

    # List of the strings that is used to add correct label for each box.
    PATH_TO_LABELS = os.path.join('object_detection', 'data', 'mscoco_label_map.pbtxt')

    NUM_CLASSES = 10 #90

    # Load label map
    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)

    track(model_id, disk_path, day_directories)


Loading frozen model: object_detection/ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb
Done


HBox(children=(IntProgress(value=0, description='Days:', max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, description='2015-01-03 00:00:00 progress:'), HTML(value='')))

KeyboardInterrupt: 

In [52]:
len(detection_records_all[0][0])

IndexError: list index out of range

In [83]:
classes


NameError: name 'classes' is not defined

In [35]:
def summarize_times(records):
    if len(records) > 1:
        for record in records:
            total = 0.
            for time in record:
                total += time.microseconds/1000000
                total += time.seconds
            print(total / len(record))
    else:
        total = 0.
        for time in records:
            total += time.microseconds/1000000
            total += time.seconds
        print(total / len(records))
    

In [54]:
detection_times_all

[]

In [53]:
summarize_times(detection_times_all)

ZeroDivisionError: float division by zero

'sd_mobilenet_v1_coco' : 'ssd_mobilenet_v1_coco_2018_01_28',
'sd_mobilenet_v2_coco' : 'ssd_mobilenet_v2_coco_2018_03_29',
'ssdlite_mobilenet_v2_coco' : 'ssdlite_mobilenet_v2_coco_2018_05_09',
'ssd_inception_v2_coco' : 'ssd_inception_v2_coco_2018_01_28',
'faster_rcnn_inception_v2_coco' : 'faster_rcnn_inception_v2_coco_2018_01_28',
'faster_rcnn_resnet50_coco' : 'faster_rcnn_resnet50_coco_2018_01_28',
            
3.51341744
5.644824460000002
4.08255038
6.199835679999998
9.05841078

within session:

0.18028956249999994
0.32623807812499994
0.16248326562499996
0.426629359375
2.671003390625

v small:

0.17886360937500004
0.32321753125
0.18154984375000005
0.3944962343749999
2.41329240625

no resize:
0.20466068749999997
0.2902670468750001
0.2412987656249999
0.43082529687500004
2.2224505624999997

In [ ]:
trigger_records_all

In [ ]:
detection_records_all

In [ ]:
ops = sess.graph.get_operations()

In [ ]:
for op in ops:
    print(op.name)

In [ ]:
trigger_record

In [ ]:
for r in trigger_record:
    for c, s in zip(r[1][2], r[1][1]):
        print('{}: {}% at {}'.format(category_index[c]['name'], s, r[0]))

In [ ]:
import pandas as pd

In [ ]:
motion_detections = pd.DataFrame({'timestamp':trigger_record})

In [ ]:
start_time = min(motion_detections.timestamp).replace(hour=0, minute=0, second=0).to_pydatetime()
end_time = max(motion_detections.timestamp).replace(hour=0, minute=0, second=0).to_pydatetime() + timedelta(days=1)

In [ ]:
time_range = pd.date_range(start=start_time, end=end_time, freq='H')
time_range_counter = pd.DataFrame({'timestamps':all_range, 'counter':0}).set_index('timestamps')

In [ ]:
for detection in motion_detections.timestamp:
    detection_time = detection.replace(minute=0, second=0)
    time_range_counter.counter[detection_time] += 1

In [ ]:
my_colors = 'b'
time_range_counter.plot(kind='bar',
             color=my_colors,
             alpha=0.5,
             figsize=(16,8),
             title='Motion Detections')


In [ ]:
hour_motion_detections = motion_detections.groupby(motion_detections.timestamp.dt.hour).count()

In [ ]:
my_colors = 'r'
hour_motion_detections.plot(kind='bar',
             color=my_colors,
             alpha=0.5,
             figsize=(16,8),
             title='Av. Motion Detections')

In [ ]:
boxes

In [ ]:
boxes.shape

In [ ]:
scores.shape

In [ ]:
classes.shape

In [ ]:
num.shape

In [ ]:
classes

In [ ]:
category_index[88.]['name']

In [ ]:
len(category_index)

In [ ]:
for i in range(1,len(category_index)+1):
    try: 
        print(category_index[i]['name'])
    except:
        pass

In [82]:
category_index

{1: {'id': 1, 'name': 'person'},
 2: {'id': 2, 'name': 'bicycle'},
 3: {'id': 3, 'name': 'car'},
 4: {'id': 4, 'name': 'motorcycle'},
 5: {'id': 5, 'name': 'airplane'},
 6: {'id': 6, 'name': 'bus'},
 7: {'id': 7, 'name': 'train'},
 8: {'id': 8, 'name': 'truck'},
 9: {'id': 9, 'name': 'boat'},
 10: {'id': 10, 'name': 'traffic light'}}